# 2회 기출 변형

- 작업 3유형 통계 반영
- 광고 두번씩 눌러주세요 

[![Hits](https://hits.seeyoufarm.com/api/count/incr/badge.svg?url=https://www.datamanim.com/dataset/practice/ex2.html&count_bg=%238921C0&title_bg=%23555555&icon=&icon_color=%23E7E7E7&title=hits&edge_flat=false)](https://hits.seeyoufarm.com)

# 작업 1유형

```{attention} 
데이터설명 : 보스턴집값, 각 행은 지역구별 집값관련된 메타정보 : https://www.kaggle.com/datasets/arunjathari/bostonhousepricedata   
**DataUrl = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e2_p1_1.csv**
```

In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e2_p1_1.csv')
df.head(2)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,NaN,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.9,4.98,24.0
1,0.02731,NaN,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.9,9.14,21.6


```{admonition} 1-1
주어진 Dataset에서 CRIM값이 가장 큰 10개의 지역을 구하고     
10개의 지역의 CRIM값을 그 중 가장 작은 값으로 대체하라. 그리고          
AGE 컬럼 값이 80이상인 대체 된 CRIM 평균값을 구하라      
```

In [3]:
temp = df['CRIM'].sort_values(ascending=False).head(10)
df.loc[df.index.isin(temp.index), 'CRIM'] = temp.sort_values().values[0]
df.loc[df['AGE'] >= 80, 'CRIM'].mean()

5.759386625

5.759386625

```{admonition} 1-2
**1-1에서 사용한 데이터에서 RM 중앙값으로 해당 컬럼의 결측치를 대체하라          
그리고 해당 컬럼의 결측치 대치 전후의 표준편차 차이의 절댓값을 소숫점 이하 3째자리 까지 구하라**
```

In [6]:
v1 = df['RM'].std()
v2 = df['RM'].fillna(df['RM'].median()).std()
print(f'{abs(v2 - v1):.3f}')

0.027


0.027

```{admonition} 1-3
**주어진 Dataset의 DIS 평균으로 부터 1.5 * 표준편차를 벗어나는 영역을 이상치라고 판단하고     
DIS 컬럼의 이상치들의 합을 구하여라.**
```

In [10]:
m, s = df['DIS'].mean(), df['DIS'].std()
df.loc[(df['DIS'] < (m - 1.5 * s)) | (df['DIS'] > (m + 1.5 * s)), 'DIS'].sum()

404.4101

404.4101

# 작업 2유형

```{attention}  
데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)    
x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv    
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv  
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv    
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv    
데이터 출처 :https://www.kaggle.com/datasets/prachi13/customer-analytics (참고, 데이터 수정)       

**x_train 데이터로 학습한 모델을 x_test에 적용하여 예측한 결과를 제출하라. 평가 지표는 f1_score이다.**
```  

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [61]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score

In [62]:
def get_model_score(model, xtrain, xtest, ytrain, ytest):
    A1 = model.score(xtrain, ytrain)
    A2 = model.score(xtest, ytest)
    
    ypred = model.predict(xtest)
    B = f1_score(ytest, ypred)
    C = precision_score(ytest, ypred)
    D = recall_score(ytest, ypred)
    
    ypred1 = model.predict_proba(xtest)[:,1]
    E = roc_auc_score(ytest, ypred1)
    
    return pd.Series([A1, A2, B, C, D, E], index='Train_Accuracy Test_Accuracy F1 Precision Recall ROC'.split())

In [40]:
def compare_model(xtrain, xtest, ytrain, ytest):
    result = pd.DataFrame()
    
    model = LogisticRegression().fit(xtrain, ytrain)
    result[f'{model.__class__.__name__}'] = get_model_score(model, xtrain, xtest, ytrain, ytest)
    
    model = KNeighborsClassifier().fit(xtrain, ytrain)
    result[f'{model.__class__.__name__}'] = get_model_score(model, xtrain, xtest, ytrain, ytest)
    
    model = DecisionTreeClassifier(random_state=1111).fit(xtrain, ytrain)
    result[f'{model.__class__.__name__}'] = get_model_score(model, xtrain, xtest, ytrain, ytest)
    
    model = RandomForestClassifier(random_state=1111).fit(xtrain, ytrain)
    result[f'{model.__class__.__name__}'] = get_model_score(model, xtrain, xtest, ytrain, ytest)
    
    model = XGBClassifier().fit(xtrain, ytrain)
    result[f'{model.__class__.__name__}'] = get_model_score(model, xtrain, xtest, ytrain, ytest)
    
    result = result.T
    result.insert(2, 'DIFF', (result['Train_Accuracy']-result['Test_Accuracy']).abs())
    return result

In [42]:
def get_model_data(dfX, Y, train_size, x_features, y_feature, is_normalized=False):
    X_temp = dfX[x_features]
    YF = Y[y_feature]
    
    if is_normalized:
        X_temp = MinMaxScaler().fit_transform(X_temp)
        
    XF = X_temp[:train_size]
    X_submissionF = X_temp[train_size:]
    
    return XF, X_submissionF, YF

In [14]:
pd.options.display.max_rows = 2000
pd.options.display.max_columns = 25
pd.options.display.float_format = '{:,.4f}'.format

In [15]:
X = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv')
Y = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv')
X_submission = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv')
Y_actual = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv')

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gms        6598 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 567.1+ KB


In [17]:
X_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401 entries, 0 to 4400
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   4401 non-null   int64 
 1   Warehouse_block      4401 non-null   object
 2   Mode_of_Shipment     4401 non-null   object
 3   Customer_care_calls  4401 non-null   object
 4   Customer_rating      4401 non-null   int64 
 5   Cost_of_the_Product  4401 non-null   int64 
 6   Prior_purchases      4401 non-null   int64 
 7   Product_importance   4401 non-null   object
 8   Gender               4401 non-null   object
 9   Discount_offered     4401 non-null   int64 
 10  Weight_in_gms        4401 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 378.3+ KB


In [18]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   6598 non-null   int64
 1   Reached.on.Time_Y.N  6598 non-null   int64
dtypes: int64(2)
memory usage: 103.2 KB


In [19]:
dfX = pd.concat([X, X_submission], axis=0, ignore_index=True)

In [20]:
dfX.isna().sum().sum()

0

In [36]:
for feature in dfX.select_dtypes(include='object').columns:
    print(f'{feature} : {dfX[feature].unique()}')

In [26]:
dfX['Customer_care_calls'] = dfX['Customer_care_calls'].replace(r'\$', '', regex=True).astype('int')

In [29]:
dfX['Gender'] = dfX['Gender'].replace(['F', 'M'], [0, 1])

In [32]:
dfX['Warehouse_block'] = LabelEncoder().fit_transform(dfX['Warehouse_block'])

In [35]:
features = ['Mode_of_Shipment', 'Product_importance']
temp = dfX.drop(columns=features)
temp_list = [temp]

for feature in features:
    temp = pd.get_dummies(dfX[feature])
    temp_list.append(temp)
    
dfX = pd.concat(temp_list, axis=1)

In [38]:
dfX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   10999 non-null  int64
 1   Warehouse_block      10999 non-null  int64
 2   Customer_care_calls  10999 non-null  int64
 3   Customer_rating      10999 non-null  int64
 4   Cost_of_the_Product  10999 non-null  int64
 5   Prior_purchases      10999 non-null  int64
 6   Gender               10999 non-null  int64
 7   Discount_offered     10999 non-null  int64
 8   Weight_in_gms        10999 non-null  int64
 9   Flight               10999 non-null  uint8
 10  Road                 10999 non-null  uint8
 11  Ship                 10999 non-null  uint8
 12  high                 10999 non-null  uint8
 13  low                  10999 non-null  uint8
 14  medium               10999 non-null  uint8
dtypes: int64(9), uint8(6)
memory usage: 837.9 KB


In [46]:
x_features = ['Warehouse_block', 'Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Gender', 'Discount_offered', 'Weight_in_gms', 'Flight', 'Road', 'Ship', 'high', 'low', 'medium']
y_feature = 'Reached.on.Time_Y.N'

In [47]:
XF, X_submissionF, YF = get_model_data(dfX, Y, len(X), x_features, y_feature, is_normalized=False)

In [48]:
xtrain, xtest, ytrain, ytest = train_test_split(XF, YF, test_size=0.3, random_state=1111, stratify=YF)

In [64]:
result = compare_model(xtrain, xtest, ytrain, ytest)

In [65]:
result.sort_values(['Train_Accuracy'], ascending=[False])

,Train_Accuracy,Test_Accuracy,DIFF,F1,Precision,Recall,ROC
DecisionTreeClassifier,1.0000,0.6449,0.3551,0.7022,0.7025,0.7019,0.6313
RandomForestClassifier,1.0000,0.6601,0.3399,0.6889,0.7587,0.6308,0.7379
XGBClassifier,0.9628,0.6561,0.3067,0.6985,0.7319,0.6681,0.7426
KNeighborsClassifier,0.7806,0.6596,0.1210,0.7049,0.7298,0.6816,0.7385
LogisticRegression,0.6397,0.6414,0.0017,0.6934,0.7075,0.6799,0.7138


In [53]:
model = DecisionTreeClassifier(random_state=1111).fit(xtrain, ytrain)
print(get_model_score(model, xtrain, xtest, ytrain, ytest))

Train_Accuracy   1.0000
Test_Accuracy    0.6449
F1               0.7022
dtype: float64


In [54]:
Y_pred = model.predict(X_submissionF)

In [56]:
submission = pd.DataFrame({'ID': X_submission['ID'], y_feature: Y_pred})

In [57]:
submission.to_csv('./01029120956.csv', index=False)

In [58]:
Y_predict = pd.read_csv('./01029120956.csv')

In [59]:
print(f1_score(Y_actual[y_feature], Y_predict[y_feature]))

0.6966378482228626


```
train accuracy : 1.0
validation accuracy : 0.6395775941230487

train f1_score : 1.0
validation f1_score : 0.6744089589382

train recall_score : 1.0
validation recall_score : 0.630721489526765

train precision_score : 1.0
validation precision_score : 0.7245989304812834

train auc : 1.0
validation auc : 0.7261997118475008
````

# 작업 3유형

```{attention} 
어느 호수에서 잡은 물고기 122마리 길이 데이터(자체제작)
**DataUrl = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e2_p3_1.csv**
```

In [66]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e2_p3_1.csv')
df.head()

,fish,height
0,fish_1,33.74cm
1,fish_2,31.64cm
2,fish_3,27.49cm
3,fish_4,31.43cm
4,fish_5,27.56cm


```{admonition} 3-1-a
**122마리의 height 평균값을 m(미터) 단위로 소숫점 이하 5자리까지 실수 값만 출력하라**
```

In [74]:
df['height2'] = df['height'].replace(r'cm', '', regex=True).astype('float') / 100
print(round(df['height2'].mean(), 5))

0.29951


0.29951

```{admonition} 3-1-b
**모집단의 평균 길이가 30cm 인지 확인하려 일표본 t 검정을 시행하여 확인하려한다. 검정통계량을 소숫점 이하 3째자리까지 구하여라**
```

In [75]:
from scipy.stats import ttest_1samp

In [77]:
tvalue, pvalue = ttest_1samp(df['height2'], 0.3, alternative='two-sided')
print(f'{tvalue:.3f}')

-0.217


-0.217

```{admonition} 3-1-c
**위의 통계량에 대한 p-값을 구하고 (반올림하여 소숫점 이하 3째자리), 유의수준 0.05하에서 귀무가설과 대립가설중 유의한 가설을 하나를 선택하시오(귀무/대립)**
```

In [79]:
print(f'{pvalue:.3f}')
print(f'{"대립" if pvalue < 0.05 else "귀무"}')

0.829
귀무


```
0.829
귀무
````


```{attention} 
조사결과 70%의 성인 남성이 3년 동안에 적어도 1번 치과를 찾는다고 할때, 21명의 성인 남성이 임의로 추출되었다고 하자. 
```

```{admonition} 3-2 - a
**21명 중 16명 미만이 치과를 찾았을 확률(반올림하여 소숫점 이하 3자리)**
```

In [44]:
from scipy.stats import binom

n = 21
p = 0.7
k = 16

# P(X < k) 계산
prob = binom.cdf(k-1, n, p)
print(round(prob, 3))

0.637


In [99]:
from scipy.stats import binom

binom(n=21, p=0.7).cdf(16-1)
1 -binom(n=21, p=0.7).cdf(18)

0.027129370541188025

```{admonition} 3-2 - b
**적어도 19명이 치과를 찾았을 확률(반올림하여 소숫점 이하 3자리)**
```

In [92]:
from scipy.stats import binom

n = 21
p = 0.7
k = 19

# P(X >= k) 계산
prob = 1 - binom.cdf(k-1, n, p)
print(round(prob, 4))

0.0271
